# SPAN-I Field-of-View Diagnostic
Greetings! Below is a guide to qualitatively assessing the field-of-view (FOV) of the PSP SPAN-I instrument, and subsequent interpretation on data reliability. We note that the SPAN-I instrument has many caveats and the data remains preliminary due to ongoing calibration work done by the instrument team. 

For optimal scientific interpretation of noteworthy event, the reader is strongly encouraged to contact a member of the instrument team:<br>
Roberto Livi rlivi@berkeley.edu <br>
Ali Rahmati rahmati@berkeley.edu <br>
Michael McManus mdmcmanus@berkeley.edu <br>
Davin Larson davin@berkeley.edu <br>

This tutorial was brought to you by: <br>
Jaye Verniero jaye.l.verniero@nasa.gov <br>
Kristoff Paulson kristoff.paulson@cfa.harvard.edu <br>
who may serve as points of contact for this tutorial. <br>

The paper describing the SPAN-I instrument in more detail can be found here: <br>https://doi.org/10.3847/1538-4357/ac93f5

Unlike SPC, the SPAN-I sensor is located behind the TPS heat shield and does not have a direct view of the sun line. That being said, when PSP approaches closer to the Sun, the speed of the spacecraft increases and so does SPAN-I's aberration angle. Therefore, there are times when SPAN-I measures the full proton VDF but we have to be a careful. This walkthrough is intended to teach the data user how to investigate the reliability of SPAN-I's Field-of-View (FOV). The three main culprits to check are (1) Phi angle coverage, (2) Density comparison to Quasithermal Noise (QTN), and (3) magnetic variability. There are other anamolies caused by the theta coverage at high energies. The deflection coverage is reduced at energies above 5 keV (which so far, only effects E13).

Check here for updated data release notes.
http://sweap.cfa.harvard.edu/Data.html

In [ ]:
#install packages
!pip install wget
%pip install --update pyspedas

In [ ]:
import pyspedas
from pytplot import tplot, store_data, get_data ,tlimit,xlim,ylim,tplot_options,options,split_vec,cdf_to_tplot,divide,tplot_names,get_timespan
from pyspedas import time_string, time_double
import wget
from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)
import cdflib

#import math functions 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import dateutil.parser
import matplotlib.pyplot as plt

First, we download data:

In [ ]:
#specify time range in the form ['yyyy-mm-dd/hh:mm:ss','yyyy-mm-dd/hh:mm:ss']
trange=['2020-01-23','2020-02-03']

#specify data type to plot
datatype='spi_sf00_l3_mom' #protons
spi_vars = pyspedas.psp.spi(trange=trange, datatype=datatype, level='l3', time_clip=True)


## 1.) Phi Angle Coverage
The first (and perhaps most dominant) effect on SPAN-I data reliabiliy is how much the plasma is cut off by the heat shield. This is mostly in the phi direction of the instrument. To see this, we plot the energy flux in the three planes of the instrument: Energy, Theta (anode), and Phi (look direction). 

In [ ]:
prefix='psp_spi_'
tplot([prefix+'SUN_DIST',prefix+'EFLUX_VS_ENERGY',prefix+'EFLUX_VS_THETA',prefix+'EFLUX_VS_PHI'])

In the fourth panel, we see how much energy flux is captured in each $\phi$ angle bin. To determine "good" coverage by eye, one should observe that there exists a peak in the range of angles, meaning the peak (or core) of the distribution is covered in the phi measurement plane. We also want to make sure this peak is located at the 160 degree mark or lower. Note that the 180 bin will always appear to have lower flux, which may be a deceptive peak.


Anytime, the collapsed VDFs (E, theta, phi), approach the edges of the coverage, we have partial loss of VDF information. Theta and Energy coverage are also an important factor to consider, especially in later encounters.

In [ ]:
#define variables
eflux_phi_data=get_data(prefix+'EFLUX_VS_PHI')
times_unix=eflux_phi_data.times
eflux = eflux_phi_data.y
phi = eflux_phi_data.v

#determine phi angle with max eflux
max_phi_ind = np.argmax(eflux, axis=1)
max_phi = phi[0, max_phi_ind]
print(max_phi)

#determine average phi
avg_phi = np.average(eflux,axis=1)
print(avg_phi)
print(avg_phi.shape)

In [ ]:
times = [dateutil.parser.parse(t) for t in time_string(times_unix)]

fig, ax = plt.subplots(figsize=(12, 5))
p = ax.pcolormesh(times[::100], phi[0, :], eflux[::100].T, norm=matplotlib.colors.LogNorm())
plt.colorbar(p, ax=ax, label=f'$(cm^2 \\ s \\ sr \\ eV)^{-1}$')
ax.plot(times, max_phi, 'k')
ax.set(ylim=(140, 170), xlabel='Time', ylabel=f"$\\phi$ [deg]");

In [ ]:
#define fov array
tlen = times_unix.shape[0]
phi_fov=np.ones(tlen)

#set threshold 163.125 degrees
phi_thresh = phi[0,1]

#set fov to 0, meaning reliable, if less than 163.125
for x in range(0,tlen):
    if max_phi[x] < phi_thresh:
        phi_fov[x]=0
        


Now we have array of zeros and ones indicating if the phi coverage is sufficient. We can mark the insufficient times with red xs below.

In reality, this determination is not quantized or binary. This is a 0th order, HIGHLY rudimentary diagnostic. 

In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))

#print(times.shape)
start_tind = 66500
stop_tind = 67000
p = ax.pcolormesh(times[start_tind:stop_tind], phi[0,:], eflux[start_tind:stop_tind].T, norm=matplotlib.colors.LogNorm())
plt.colorbar(p, ax=ax, label=f'$(cm^2 \\ s \\ sr \\ eV)^{-1}$')
ax.plot(times[start_tind:stop_tind], max_phi[start_tind:stop_tind], 'k')

ax.plot(times[start_tind:stop_tind],phi_fov[start_tind:stop_tind]*160,'r+')    
        
ax.set(ylim=(140, 170), xlabel='Time', ylabel=f"$\\phi$ [deg]");


# Quasithermal Noise Comparison 

The Quasithermal Noise (QTN) data from the FIELDS instrument is presently the most reliable measurement of density. See Moncuquet et al. (DOI: 10.3847/1538-4365/ab5a84) for more details.

 Since SPAN-I measures partial moments of the VDF, density is highly sensitive to inaccuracies. In addition to the $\phi$ angular coverage assessment described above, the data user is encouraged to compare the SPAN-I measured density to the density derived from QTN. Note that this measurement is also prone to error, and there is currently no agreed upon value of density. Ongoing calibration efforts by the instrument team continue to converge closer to an answer. Below, we show how to compare the QTN and SPAN-I measured density.

Note there is not a defined threshold for offsets, as the answer depends on the physical problem at hand. The user is encouraged to reach out to the SPAN-I instrument team for recommendations if the SPAN-I density in the time period of interest deviates strongly from the QTN.

Future additions to this tutorial will show how to also compare with density measured by SPC.


In [ ]:
#load data with pySPEDAS
#note this load routine does not currently work
datatype = 'sqtn_rfs_v1v2'
vars_qtn = pyspedas.psp.fields(trange=trange, datatype=datatype, level='l3', time_clip=True)
print(vars_qtn)

Alternatively, we can instead use cdflib and a handy routine 'cdf_to_tplot', which converts variable in cdf files to tplot variables.

In [ ]:
from datetime import datetime

year=2020
month=1
day=29

user_datetime = datetime(year,month,day)

def yyyymmdd(dt) : return f"{dt.year:04d}{dt.month:02d}{dt.day:02d}"

#Import from file directory
QTNfile_directoryRemote = f'https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l3/sqtn_rfs_v1v2/{user_datetime.year:04d}/'
QTNfile_filename = f'psp_fld_l3_sqtn_rfs_v1v2_{yyyymmdd(user_datetime)}_v1.0.cdf'

import os.path

#check if file is already downloaded. If so, skip download. If not, download in local directory.
if os.path.isfile(QTNfile_filename):
    print(f"File already exists in local directory - [{QTNfile_filename}]")
    QTNfile = QTNfile_filename
else:
    print("File doesn't exist. Downloading ...")
    QTNfile = wget.download(QTNfile_directoryRemote + QTNfile_filename)

In [ ]:
QTN_dat = cdflib.CDF(QTNfile)
#print variable names in cdf file
print(dat._get_varnames())
#check variable formats in cdf file
print(QTN_dat)

In [ ]:
#convert variables in cdf files to tplot variables for quick plotting
cdf_to_tplot(QTNfile)

In [ ]:
#specify shorter time range in format 'yyyy-mm-dd hh:mm:ss'
trange1=['2020-01-29 00:00:00','2020-01-30 00:00:00']
#set limits 
xlim(trange1[0],trange1[1])
help,options
options(prefix+'DENS','ylog',0)
ylim(prefix+'DENS',0,1500)
ylim('electron_density',0,1500)
tplot(['electron_density',prefix+'DENS'])

In [ ]:
#overlay both variables on the same axis by defining a new tplot variable
store_data("dens_compare",data=['electron_density',prefix+'DENS'])

#set legend
options('dens_compare', 'color', ['blue', 'red'])
options('dens_compare', 'legend_names', ['QTN', 'SPAN-I'])
tplot('dens_compare')

The 'divide' function divides two tplot variables and is equivalent 'div_data' in IDL. It is useful since it automatically interpolates the two variables if they have different time cadences.

We can see what happens when we divide QTN by SPAN-I density and assess the deviation from 1. 

In [ ]:
divide('electron_density',prefix+'DENS',new_tvar='dens_div')
ylim('dens_div',0,3)
options('dens_div','ytitle','QTN/SPAN-I Dens')
tplot('dens_div')

As evident in the above two plots, the SPAN-I density deviates from QTN roughly in the period between 09:00 - 11:00. Thus, if a data user sees this level of discrepency in a time period of interest, extra support should be sought from the instrument team.

# Magnetic Field Variability

A third known culprit of questionable data reliability is due to high magnetic field variability. The SPAN-I instrument samples the plasma at a much lower cadence than the FIELDS magnetometer samples the magnetic field. It is therefore important to assess how much the instanteous magnetic field changes with respect to the averaged magnetic field over the lower SPAN-I time cadence. If the magnetic field is changing at a rapid enough timescale, measurements could be skewed and the user is again encouraged to reach out to the instrument team. As before, the SPAN-I team cannot recommend a threshold value for this variability, as it depends on the physical problem. Below, we show how to calculate the angular fluctuations in the magnetic field during the SPAN-I distribution integration time. As described by Eq. 4 in Woodham et al. 2021 (DOI: 10.1051/0004-6361/202039415), this is quantified by:

$\psi_B = \frac{1}{N}\Sigma_{i=1}^N arccos(\mathbf{\hat{B}}_i \cdot \mathbf{\hat{B}}_{SPAN})$ 

 where $N$ is the number of instantaneous $\mathbf{\hat{B}}$ field measurments during one SPAN-I integration time, $\mathbf{\hat{B}}_{SPAN}$ is the average magnetic field direction, and $\mathbf{\hat{B}}_i$ is the instantaneous magentic field unit vector.
 
 Qualitative assessment of this parameter is demonstrated so the user can gain more intuition on the nuances of interpreting partial plasma moments.

In [ ]:
#specify time range in the form ['yyyy-mm-dd/hh:mm:ss','yyyy-mm-dd/hh:mm:ss']
#time_range=['2021-04-29/05:00:00','2021-04-29/12:00:00']

#load data
#note we are downloading lower time cadence, 4 samples per cycle data
#the full cadence is not necessary and would take time to download
fields_vars=pyspedas.psp.fields(datatype='mag_sc_4_sa_per_cyc',trange=trange,time_clip=True)
print(trange)
#plot data vs time
tplot('psp_fld_l2_mag_SC_4_Sa_per_Cyc')
get_timespan('psp_fld_l2_mag_SC_4_Sa_per_Cyc')

In [ ]:
span_b=get_data(prefix+'MAGF_SC')
tspan = span_b.times

hires_b=get_data('psp_fld_l2_mag_SC_4_Sa_per_Cyc')
thr = hires_b.times
bhr = hires_b.y

import bisect
from pyspedas import time_datetime
ts_dt=time_datetime(tspan)
thr_dt=time_datetime(thr)


In [ ]:
N = np.size(tspan)

alpha_mean=np.zeros(N)
for i in range(0,N-2):

    tr = [ts_dt[i],ts_dt[i+1]]
    #find time indices of B in one span accumulation time
    tind_start=bisect.bisect_left(thr_dt,tr[0])
    tind_stop=bisect.bisect_left(thr_dt,tr[1])

    B0 = bhr[tind_start,:]
    B_mag = np.sqrt(B0[0]**2 + B0[1]**2 + B0[2]**2)
    B0_hat = B0/B_mag

    bt_len = tind_stop-tind_start #length new B time array
    cos_alpha = np.zeros(bt_len-1)
    
    
    for j in range(1,bt_len-1):
        Bi = bhr[j+tind_start,:]
        Bi_mag = np.sqrt(Bi[0]**2 + Bi[1]**2 + Bi[2]**2)
        cos_alpha[j-1]=np.sum(B0_hat * Bi)/Bi_mag
    print(i)
    alpha = np.arccos(cos_alpha)*180/np.pi
    alpha_mean[i]=np.mean(alpha)



In [ ]:
print(alpha_mean.shape)


In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))

times_tspan = [dateutil.parser.parse(t) for t in time_string(tspan)]

ax.plot(times_tspan, alpha_mean, 'k')
ax.set( xlabel='Time', ylabel='mean $\psi$');